In [36]:
ruta = r"C:\Program Files\Computers and Structures\ETABS 20\CSI API ETABS v1.chm"
os.startfile(ruta)

# Librerias

In [1]:
#                                                   LIBRARIES
import os
import sys
import comtypes.client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#                                               INIT INSTANCE
helper = comtypes.client.CreateObject('ETABSv1.Helper')
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)

# myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject")     # Abrir una nueva ventana de Etabs
myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject")            # Trabajar con una ventana abierta de Etabs

# myETABSObject.ApplicationStart()

SapModel = myETABSObject.SapModel               # Crear un objeto SapModel

#SapModel.InitializeNewModel()                   # Iniciar Modelo

#SapModel.File.NewBlank()                        # Crear un modelo en blanco

In [40]:
#                                                   INSERT UNITS
kgf_m_C = 8
SapModel.SetPresentUnits(kgf_m_C)

MATERIAL_CONCRETE = 2
SapModel.PropMaterial.SetMaterial('CONCRETO280', MATERIAL_CONCRETE)
E = 15000*(280**0.5)*10000   # (kgf/m2)
SapModel.PropMaterial.SetMPIsotropic('CONCRETO280', E, 0.2, 9.90E-06)
SapModel.PropMaterial.SetWeightAndMass('CONCRETO280', 1, 2400)
SapModel.PropMaterial.SetOConcrete('CONCRETO280', 2800000, False, 0, 1, 2, 0.0022, 0.0052)

MATERIAL_REBAR = 6
SapModel.PropMaterial.SetMaterial('REBAR', MATERIAL_REBAR)
SapModel.PropMaterial.SetMPIsotropic('REBAR', 2.0E10, 0.2, 9.90E-06)
SapModel.PropMaterial.SetWeightAndMass('REBAR', 1, 7850)
SapModel.PropMaterial.SetORebar("REBAR", 42000000, 63000000, 46000000, 69000000, 2, 2, 0.02, 0.1, False)

0

In [41]:
#                                                   DEFINE: FRAMES
# FrameObj Interface
# Definir las propiedades de Seccion a un Frame rectangular
SapModel.PropFrame.SetRectangle('R1',               # Nombre del nuevo frame
                                'CONCRETO280',      # Nombre del Material, debe existir
                                0.6,                # peralte
                                0.3 )               # base
                                
# Datos extra para Frame Vigas        
SapModel.PropFrame.SetRebarBeam("R1",               # Nombre de un frame existente
                                "REBAR",            # Material del acero longitudinal
                                "REBAR",            # Material del acero transversal
                                0.06,               # Recubrimiento superior   6cm
                                0.06,               # Recubrimiento inferior   6cm
                                1.29*10**-4,        # Top Left Area
                                1.29*10**-4,        # Top Right Area
                                1.29*10**-4,        # Bottom Left Area
                                1.29*10**-4)        # Bottom Right Area

# # Añadir objeto Frame por Coordenadas
# FrameName1 = ' '
# [FrameName1, _] = SapModel.FrameObj.AddByCoord(0, 0, 0,      # Coordenadas X,Y,Z del punto inicial
#                                               0, 0, 10,      # Coordenadas X,Y,Z del punto final
#                                               FrameName1,    # Vacio, el programa asignará un nombre por defecto
#                                               'R1')          # Nombre de la seccion, debe existir el nombre de esa seccion, opcional

# FrameName2 = ' '
# FrameName3 = ' '
# [FrameName2, ret] = SapModel.FrameObj.AddByCoord(0, 0, 10, 8, 0, 16, FrameName2, 'R1')
# [FrameName3, ret] = SapModel.FrameObj.AddByCoord(-4, 0, 10, 0, 0, 10, FrameName3, 'R1')

0

In [42]:
#                                                       DEFINE: SLABS
SapModel.PropArea.SetSlab("Losa Macisa",          # Nombre de la Losa
                          0,                      # Slab Type - Slab
                          1,                      # ShellType, ShellThin
                          "CONCRETO280",          # Material
                          0.15)                     # Thickness  

#SapModel.PropArea.SetSlab("Slab Ribbed2", 3, 1, "CONCRETO280", 0.20)
#SapModel.PropArea.SetSlabRibbed("Slab Ribbed2", 0.20, 0.05, 0.1, 0.1, 0.4, 1)
# # Definir las propiedades de la Losa Slab
# SapModel.PropArea.SetSlab("Slab Ribbed",          # Nombre de la Losa
#                           3,                      # Slab Type - Ribbed
#                           3,                      # ShellType, ShellThin
#                           "CONCRETO280",          # Material
#                           0.15)                   # Thickness  

# # Definir las propiedades de la Losa Ribbed
# SapModel.PropArea.SetSlabRibbed("Slab Ribbed",   # Nombre de la Losa
#                                  11.1,           # OverallDepth              
#                                  22.2,           # SlabThickness
#                                  33.3,           # StemWidthTop
#                                  44.4,           # StemWidthBottom
#                                  55.5,           # RibSpacing
#                                  2)              # RibsParallelTo - Local Axis

0

In [43]:
#                                               DEFINE: WALLS
SapModel.PropArea.SetWall("Placa",            # Name
                          1,                  # WallPropType , Seccion Especificada por el usuario
                          1,                  # Shell Type - Shell Thin   
                          "CONCRETO280",      # Nombre del Material
                          0.20)               # Thickness

# MyValue = [1, 1, 1, 1, 1, 1, 1, 1, 1]
# SapModel.PropArea.SetModifiers("Placa", MyValue)

AreaName = ' '
x = [0,2,2,0]
y = [0,0,0,0]
z = [0,0,3,3]
SapModel.AreaObj.AddByCoord(4, # Numero de puntos que definen el area
                            x, # Coordenadas X
                            y, # Coordenadas Y
                            z, # Coordenadas Z
                            AreaName, # Vacio, el programa asignará un nombre por defecto
                            "Placa")  # Nombre de la seccion, debe existir el nombre de esa seccion, opcional
SapModel.View.RefreshView(0,          # 0: Refresh todas las ventanas
                        False)        # False: No matener el zoom

0

In [46]:
#                                                                   EXAMPLE

L = 7   # m
b = 7   # m
B = 7  # m
H = 2.7 # m

FrameName1 = ' '
FrameName2 = ' '
FrameName3 = ' '
FrameName4 = ' '
FrameName5 = ' '
FrameName6 = ' '
FrameName7 = ' '
FrameName8 = ' '
# Columnas
[FrameName1, ret] = SapModel.FrameObj.AddByCoord(0, 0, 0, 0, 0, H, FrameName1, 'R1')
[FrameName2, ret] = SapModel.FrameObj.AddByCoord(B, 0, 0, B, 0, H, FrameName2, 'R1')
[FrameName3, ret] = SapModel.FrameObj.AddByCoord(0, L, 0, 0, L, H, FrameName3, 'R1')
[FrameName4, ret] = SapModel.FrameObj.AddByCoord(b, L, 0, b, L, H, FrameName4, 'R1')
# Vigas
[FrameName5, ret] = SapModel.FrameObj.AddByCoord(0, 0, H, B, 0, H, FrameName5, 'R1')
[FrameName6, ret] = SapModel.FrameObj.AddByCoord(0, 0, H, 0, L, H, FrameName6, 'R1')
[FrameName7, ret] = SapModel.FrameObj.AddByCoord(B, 0, H, b, L, H, FrameName7, 'R1')
[FrameName8, ret] = SapModel.FrameObj.AddByCoord(0, L, H, b, L, H, FrameName8, 'R1')

AreaName = ' '
x = [0,B,b,0]
y = [0,0,L,L]
z = [H,H,H,H]
SapModel.AreaObj.AddByCoord(4, # Numero de puntos que definen el area
                            x, # Coordenadas X
                            y, # Coordenadas Y
                            z, # Coordenadas Z
                            AreaName, # Vacio, el programa asignará un nombre por defecto
                            "Losa Macisa")  # Nombre de la seccion, debe existir el nombre de esa seccion, opcional

SapModel.View.RefreshView(0,          # 0: Refresh todas las ventanas
                        False)        # False: No matener el zoom

0

In [45]:
def connect_to_etabs():
    helper = comtypes.client.CreateObject('ETABSv1.Helper')
    helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
    try:
        myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject")
        print("Connected to ETABS model")
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
    SapModel = myETABSObject.SapModel
    return SapModel,myETABSObject,helper

SapModel, myETABSObject, helper = connect_to_etabs()

# Definir los parámetros necesarios para llamar al método GetTableForDisplayArray
TableKey = "Nombre_de_la_tabla"
FieldKeyList = None  # Puedes definir una lista de campos si es necesario
GroupName = ""  # Nombre del grupo, si es necesario
TableVersion = 0  # Versión de la tabla, si es necesario
FieldsKeysIncluded = None  # Puedes definir una lista de campos si es necesario
NumberRecords = 0  # Número de registros, se actualizará después de la llamada
TableData = None  # Los datos de la tabla se almacenarán aquí después de la llamada

# Llamar al método GetTableForDisplayArray
result = SapModel.GetTableForDisplayArray(TableKey, FieldKeyList, GroupName, TableVersion, FieldsKeysIncluded, NumberRecords, TableData)

# Verificar si la llamada fue exitosa
if result != 0:
    print("Error al obtener los datos de la tabla.")
else:
    # Procesar los datos de la tabla
    print("Datos de la tabla:")
    print(TableData)

# Cerrar ETABS
ETABSApp.ApplicationExit()

Connected to ETABS model


AttributeError: GetTableForDisplayArray